## Imports

In [ ]:
import plotly.express as px
import plotly.io as pio
import geopandas as gpd
import pandas as pd
import chart_studio.plotly as py

# The bigquery client will not function unless you have set your environmental variable GOOGLE_APPLICATION_CREDENTIALS with a key
from google.cloud import bigquery
client = bigquery.Client()

## Region Definitions (included in forms.py on Django website)

In [ ]:
northeast = ['US_CT', 'US_ME', 'US_MA', 'US_NH', 'US_RI', 'US_VT', 'US_DE', 'US_NJ', 'US_NY', 'US_PA']
new_england = ['US_CT', 'US_ME', 'US_MA', 'US_NH', 'US_RI', 'US_VT']
mid_atlantic = ['US_DE', 'US_NJ', 'US_NY', 'US_PA']
midwest = ['US_IL', 'US_IN', 'US_MI', 'US_OH', 'US_WI', 'US_IA', 'US_KS', 'US_MN', 'US_MO', 'US_NE', 'US_ND', 'US_SD']
midwest_east_north_central = ['US_IL', 'US_IN', 'US_MI', 'US_OH', 'US_WI']
midwest_west_north_central = ['US_IA', 'US_KS', 'US_MN', 'US_MO', 'US_NE', 'US_ND', 'US_SD']
south = ['US_DC', 'US_FL', 'US_GA', 'US_MD', 'US_NC', 'US_SC', 'US_VA', 'US_WV', 'US_AL', 'US_KY', 'US_MS', 'US_TN', 'US_AR', 'US_LA', 'US_OK', 'US_TX']
south_atlantic = ['US_DC', 'US_FL', 'US_GA', 'US_MD', 'US_NC', 'US_SC', 'US_VA', 'US_WV']
south_east_south_central = ['US_AL', 'US_KY', 'US_MS', 'US_TN']
south_west_south_central = ['US_AR', 'US_LA', 'US_OK', 'US_TX']
west = ['US_AZ', 'US_CO', 'US_ID', 'US_MT', 'US_NV', 'US_NM', 'US_UT', 'US_WY', 'US_CA', 'US_OR', 'US_WA']
mountain = ['US_AZ', 'US_CO', 'US_ID', 'US_MT', 'US_NV', 'US_NM', 'US_UT', 'US_WY']
pacific = ['US_CA', 'US_OR', 'US_WA']

## Functions for creating queries to Google BigQuery dataset and for generating figures with Plotly

In [ ]:
pop_dict = {
'US_AK': 738432,'US_AL': 4858979,'US_AR': 2978204,'US_AZ': 6828065,'US_CA': 39144818,
'US_CO': 5456574,'US_CT': 3590886,'US_DC': 705749,'US_DE': 945934,'US_FL': 20271272,
'US_GA': 10214860,'US_HI': 1431603,'US_IA': 3046355,'US_ID': 1654930,'US_IL': 12859995,
'US_IN': 6619680,'US_KS': 2911641,'US_KY': 4425092,'US_LA': 4533372,'US_MA': 6794422,
'US_MD': 6045680,'US_ME': 1328361,'US_MI': 9922576,'US_MN': 5489594,'US_MO': 6083672,
'US_MS': 2992333,'US_MT': 1032949,'US_NC': 10042802,'US_ND': 756927,'US_NE': 1896190,
'US_NH': 1330608,'US_NJ': 8958013,'US_NM': 2085109,'US_NV': 2890845,'US_NY': 19795791,
'US_OH': 11613423,'US_OK': 3911338,'US_OR': 4028977,'US_PA': 12802503,'US_RI': 1056298,
'US_SC': 4896146,'US_SD': 858469,'US_TN': 6600299,'US_TX': 27469114,'US_UT': 2763885,
'US_VA': 8382993,'US_VT': 626042,'US_WA': 7170351,'US_WI': 5771337,'US_WV': 1844128,
'US_WY': 586107}

def BuildQuery(query_type, query_param, query_state=None, query_region=None, query_date=None):
    # Read shapefile containing county geometry
    usa = gpd.read_file('C:/djangosite/djangosite/coviddashboard/2020_counties/cb_2020_us_county_500k.shp')

    # Create dataframe for specified state
    df = usa.loc[usa['STUSPS'] == query_state].sort_values(by='GEOID', ascending=True).reset_index(drop=True)

    # Determine correct sql query to return
    if query_date is None:
        if query_type == 'county':
            # Calculate number of counties for county-level data visualization
            num_counties = len(df)

            # County by Latest Date
            sql = f"""
                SELECT DISTINCT location_key, {query_param}, date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
                WHERE location_key LIKE 'US_{query_state}______' AND {query_param} IS NOT NULL
                ORDER BY date DESC
                LIMIT {num_counties}
                """
        elif query_type == 'state':
            # State by Latest Date
            sql = f"""
                SELECT DISTINCT location_key, {query_param}, date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
                WHERE location_key = 'US_{query_state}' AND {query_param} IS NOT NULL
                ORDER BY date DESC
                LIMIT 1
                """

        elif query_type == 'region':
            # Region by Latest Date
            sql = f"""
                SELECT DISTINCT location_key, {query_param}, date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
                WHERE location_key in {str(query_region).replace('[','(').replace(']',')')} AND {query_param} IS NOT NULL
                ORDER BY date DESC
                LIMIT {len(query_region)}
                """

        else:
            # Country by Latest Date
            sql = f"""
                SELECT DISTINCT location_key, {query_param}, date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
                WHERE location_key LIKE 'US___' AND {query_param} IS NOT NULL and location_key NOT IN ('US_GU', 'US_PR', 'US_VI', 'US_MP', 'US_AS')
                ORDER BY date DESC
                LIMIT 51
                """
    else:
        if query_type == 'county':
            # Calculate number of counties for county-level data visualization
            num_counties = len(df)

            # County on Specific Date
            sql = f"""
                SELECT DISTINCT location_key, {query_param}, date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
                WHERE location_key LIKE 'US_{query_state}______' AND {query_param} IS NOT NULL AND date = '{query_date}'
                ORDER BY date DESC
                LIMIT {num_counties}
                """
        elif query_type == 'state':
            # State on Specific Date
            sql = f"""
                SELECT DISTINCT location_key, {query_param}, date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
                WHERE location_key = 'US_{query_state}' AND {query_param} IS NOT NULL AND date = '{query_date}'
                ORDER BY date DESC
                LIMIT 1
                """

        elif query_type == 'region':
            # Region by Latest Date
            sql = f"""
                SELECT DISTINCT location_key, {query_param}, date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
                WHERE location_key in {str(query_region).replace('[','(').replace(']',')')} AND {query_param} IS NOT NULL AND date= '{query_date}'
                ORDER BY date DESC
                LIMIT {len(query_region)}
                """
        else:
            # Country on Specific Date
            sql = f"""
                SELECT DISTINCT location_key, {query_param}, date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
                WHERE location_key LIKE 'US___' AND {query_param} IS NOT NULL and location_key NOT IN ('US_GU', 'US_PR', 'US_VI', 'US_MP', 'US_AS') AND date = '{query_date}'
                ORDER BY date DESC
                LIMIT 51
                """
    return client.query(sql).to_dataframe().sort_values(by='location_key', ascending=True).reset_index(drop=True)

def GenerateFigure(fig_type, fig_query, fig_param, fig_state=None, normalized=False):
    # Modifies query values and labels if Normalized is True
    if normalized is True and fig_type != 'county':
        normal = ' per 100,000 people'
        if fig_type == 'state':
            fig_query[fig_param] = int((fig_query[fig_param][0]*100000)/pop_dict[fig_query['location_key'][0]])
        elif fig_type == 'region':
            for loc in range(len(fig_query)):
                fig_query[fig_param][loc] = int((fig_query[fig_param][loc]*100000)/pop_dict[fig_query['location_key'][loc]])
        elif fig_type == 'country':
            for loc in range(len(fig_query)):
                fig_query[fig_param][loc] = int((fig_query[fig_param][loc]*100000)/pop_dict[fig_query['location_key'][loc]])
    else:
        normal = ''

    if fig_type == 'county':
        # Read shapefile containing all US county geometries
        usa = gpd.read_file('C:/djangosite/djangosite/coviddashboard/2020_counties/cb_2020_us_county_500k.shp')

        # Obtain all county names, FIPS codes, and geometry
        df = usa.loc[usa['STUSPS'] == fig_state].sort_values(by='GEOID', ascending=True).reset_index(drop=True)

        # Create DataFrame for county-level data
        state_df = pd.DataFrame({
            'county': df['NAME'],
            'geoid': df['GEOID'],
            'geometry': df['geometry'],
            fig_param: fig_query[fig_param]})

        gdf = gpd.GeoDataFrame(data=state_df, geometry=state_df.geometry).set_index('county')
        fig = px.choropleth(data_frame=gdf,
                            geojson=gdf.geometry,
                            locations=gdf.index,
                            color=fig_param,
                            color_continuous_scale=[(0, "white"), (0.75, "red"), (1, '#820000')],
                            labels={fig_param: fig_param.title().replace("_", " ")},
                            projection="mercator",
                            width=1000,
                            height=800,
                            title=f'{fig_param.title().replace("_", " ")} in {fig_state} Counties on {str(fig_query["date"][0])}')
        fig.update_geos(fitbounds="locations", visible=False)
        
        # Upload to plotly
        py.plot(fig, filename='plotly figure', auto_open=True, include_plotlyjs='cdn')

    elif fig_type == 'state':
        fig = px.choropleth(data_frame=fig_query,
                            locations=[fig_state],
                            locationmode="USA-states",
                            color=fig_param,
                            labels={fig_param: fig_param.title().replace("_", " ")+normal},
                            width=1000,
                            height=800,
                            scope="usa",
                            title=f'{fig_param.title().replace("_", " ")} in {fig_state} on {str(fig_query["date"][0])}'+normal)
        fig.update_geos(fitbounds="locations", visible=False)
        fig.update_layout(coloraxis_showscale=False)
        
        # Upload to plotly
        py.plot(fig, filename='plotly figure', auto_open=True, include_plotlyjs='cdn')

    elif fig_type == 'region':
        fig = px.choropleth(data_frame=fig_query,
                            locations=[state[-2:] for state in fig_query['location_key']],
                            locationmode="USA-states",
                            color=fig_param,
                            color_continuous_scale=[(0, "white"), (0.75, "red"), (1, '#820000')],
                            labels={fig_param: fig_param.title().replace("_", " ")+normal},
                            width=1000,
                            height=800,
                            scope="usa",
                            title=f'{fig_param.title().replace("_", " ")} in specified region on {str(fig_query["date"][0])}'+normal)
        fig.update_geos(fitbounds="locations", visible=False)
        
        # Upload to plotly
        py.plot(fig, filename='plotly figure', auto_open=True, include_plotlyjs='cdn')
    else:
        fig = px.choropleth(data_frame=fig_query,
                            locations=[state[-2:] for state in fig_query['location_key']],
                            locationmode="USA-states",
                            color=fig_param,
                            color_continuous_scale=[(0, "white"), (0.75, "red"), (1, '#820000')],
                            labels={fig_param: fig_param.title().replace("_", " ")+normal},
                            width=1000,
                            height=800,
                            scope="usa",
                            title=f'{fig_param.title().replace("_", " ")} in US on {str(fig_query["date"][0])}'+normal)
        fig.update_geos(fitbounds="locations", visible=False)
        
        # Upload to plotly
        py.plot(fig, filename='plotly figure', auto_open=True, include_plotlyjs='cdn')


## User-set Parameters

In [ ]:
# Query type: county, state, region, country
q_type = 'region'

# Parameter of interest: new_confirmed, new_deceased, population, etc
param = 'cumulative_deceased'

# State: State code used for county/state level queries
state = 'FL'

# Normalized: normalizes per 100,000 people if True
normalized = False

# Region: US regions as designated by Census Bureau (eg. Northeast, Midwest, South, West...)
# Subregions are also accepted here (new_england, mid_atlantic, etc)
region = midwest

# Date in format YYYY-MM-DD
date = None

## Run the functions

In [ ]:
query = BuildQuery(q_type, param, state, region, date)
GenerateFigure(q_type, query, param, state, normalized)

## Query Type Requirements

In [ ]:
"""
'county'
param (required)
state (required)
region (not required)
date (optional)

'state'
param (required)
state (required)
region (not required)
date (optional)

'region'
param (required)
state (not required)
region (required)
date (optional)

'country'
param (required)
state (not required)
region (not required)
date (optional)
"""

## Parameter Options

In [ ]:
"""
COUNTY-LEVEL PARAMETERS:

new_confirmed
new_deceased
cumulative_confirmed
cumulative_deceased
new_persons_fully_vaccinated
cumulative_persons_fully_vaccinated

population
population_male
population_female
population_age_00_09
population_age_10_19
population_age_20_29
population_age_30_39
population_age_40_49
population_age_50_59
population_age_60_69
population_age_70_79
population_age_80_and_older
"""

In [ ]:
"""
STATE-/COUNTRY-LEVEL PARAMETERS:

new_confirmed
new_deceased
cumulative_confirmed
cumulative_deceased
new_persons_vaccinated
cumulative_persons_vaccinated
new_persons_fully_vaccinated
cumulative_persons_fully_vaccinated
new_vaccine_doses_administered
cumulative_vaccine_doses_administered

population
population_male
population_female
population_age_00_09
population_age_10_19
population_age_20_29
population_age_30_39
population_age_40_49
population_age_50_59
population_age_60_69
population_age_70_79
population_age_80_and_older

new_hospitalized_patients
cumulative_hospitalized_patients
current_hospitalized_patients
current_intensive_care_patients

new_persons_fully_vaccinated_pfizer
cumulative_persons_fully_vaccinated_pfizer
new_vaccine_doses_administered_pfizer
cumulative_vaccine_doses_administered_pfizer

new_persons_fully_vaccinated_moderna
cumulative_persons_fully_vaccinated_moderna
new_vaccine_doses_administered_moderna
cumulative_vaccine_doses_administered_moderna

new_persons_fully_vaccinated_janssen
cumulative_persons_fully_vaccinated_janssen
new_vaccine_doses_administered_janssen
cumulative_vaccine_doses_administered_janssen
"""